In [ ]:
import tensorflow as tf
import numpy as np
import os

# --- Configuration ---
# You can replace this with a larger text file (e.g., a Shakespeare corpus)
# For a quick runnable example, we'll use a small built-in text.
# To use a file, uncomment the lines below and provide the path.
# FILE_PATH = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# with open(FILE_PATH, 'r') as f:
#     text = f.read()

# Small sample text for demonstration
text = """
All the world's a stage,
And all the men and women merely players;
They have their exits and their entrances;
And one man in his time plays many parts,
His acts being seven ages. At first the infant,
Mewling and puking in the nurse's arms;
Then the whining school-boy, with his satchel
And shining morning face, creeping like snail
Unwillingly to school. And then the lover,
Sighing like furnace, with a woeful ballad
Made to his mistress' eyebrow. Then a soldier,
Full of strange oaths and bearded like the pard,
Jealous in honour, sudden and quick in quarrel,
Seeking the bubble reputation
Even in the cannon's mouth. And then the justice,
In fair round belly with good capon lined,
With eyes severe and beard of formal cut,
Full of wise saws and modern instances;
And so he plays his part. The sixth age shifts
Into the lean and slipper'd pantaloon,
With spectacles on nose and pouch on side,
His youthful hose, well saved, a world too wide
For his shrunk shank; and his big manly voice,
Turning again toward childish treble, pipes
And whistles in his sound. Last scene of all,
That ends this strange eventful history,
Is second childishness and mere oblivion,
Sans teeth, sans eyes, sans taste, sans everything.
"""

# Convert all text to lowercase for consistency
text = text.lower()

# --- Data Preprocessing ---
# Create a vocabulary of unique characters
vocab = sorted(list(set(text)))
char_to_int = {char: i for i, char in enumerate(vocab)}
int_to_char = {i: char for i, char in enumerate(vocab)}

# Print vocabulary size
print(f"Vocabulary size: {len(vocab)} unique characters.")

# Convert text to integers
encoded_text = [char_to_int[char] for char in text]

# Create training sequences
seq_length = 100  # Length of input sequences
examples_per_epoch = len(encoded_text) // (seq_length + 1)

# Create character datasets
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

# Batch sequences for training
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch and shuffle the dataset
batch_size = 64
buffer_size = 10000  # TF data will prefetch this many elements to disk
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

# --- Model Definition ---
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()

# --- Training ---
# Loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Configure checkpoints to save model weights during training
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

epochs = 20  # You might need more epochs for a larger dataset
history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

# --- Text Generation Function ---
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    """
    Generates text using the trained model.

    Args:
        model: The trained Keras model.
        start_string: The initial string to start generation from.
        num_generate: The number of characters to generate.
        temperature: A float value, controls the randomness of predictions.
                     Higher temperature (e.g., 1.0) means more creative/random.
                     Lower temperature (e.g., 0.2) means more conservative/predictable.
                     Temperature close to 0 makes it almost greedy (picks the highest probability).
    Returns:
        The generated text.
    """
    # Convert start string to numbers (integers)
    input_eval = [char_to_int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0) # Add batch dimension

    text_generated = []

    # Reset the model's state before starting generation
    # This is important for stateful RNNs
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Apply temperature to logits
        predictions = predictions / temperature

        # Sample a character from the distribution
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(int_to_char[predicted_id])

    return start_string + ''.join(text_generated)

# --- User Interaction ---
if __name__ == "__main__":
    print("\n--- Text Generation ---")
    print("Enter a starting string (e.g., 'romeo and juliet'):")
    seed_text = input().lower() # Convert input to lowercase

    while True:
        try:
            print("Enter a temperature value (e.g., 0.5 for less random, 1.0 for more random, 1.5 for very creative):")
            temperature_input = float(input())
            if temperature_input <= 0:
                print("Temperature must be greater than 0. Setting to 0.1.")
                temperature_input = 0.1
            break
        except ValueError:
            print("Invalid input. Please enter a number for temperature.")

    print("\nGenerating text...")
    generated_text = generate_text(model, start_string=seed_text, num_generate=500, temperature=temperature_input)
    print("\n--- Generated Text ---")
    print(generated_text)
    print("\n--- End of Generation ---")

    # You can also load the latest checkpoint and generate text
    # latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    # model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # For inference, batch_size is 1
    # model.load_weights(latest_checkpoint)
    # model.build(tf.TensorShape([1, None]))
    # print("\n--- Generated Text from Loaded Model ---")
    # print(generate_text(model, start_string=seed_text, num_generate=500, temperature=temperature_input))

Vocabulary size: 32 unique characters.


ValueError: Unrecognized keyword arguments passed to Embedding: {'batch_input_shape': [64, None]}